# Supervised Fine-Tuning (SFT) of BERT for Text Classification: IMDB Case Study

In [ ]:

import os
import json
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import numpy as np
from hiq.vis import print_model

# Defines a PyTorch model for sentiment classification

Initializes with a pre-trained BERT model from Hugging Face.

- Adds a dropout layer for regularization.
- Adds a linear layer mapping BERT’s hidden representation (from [CLS] token) to num_classes outputs.

In forward(), it:

- Feeds inputs into BERT to get contextual embeddings.
- Extracts the pooled output (representation of [CLS] token).
- Applies dropout.
- Passes the vector through the linear classifier to get logits.


In [2]:
class SentimentClassifier(torch.nn.Module):
    """
    Sentiment classifier using Hugging Face BERT model
    """
    def __init__(self, model_name="bert-base-cased", num_classes=2, dropout=0.1):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = torch.nn.Dropout(dropout)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        # Get BERT outputs
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        # Use [CLS] token representation for classification
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

## Model Architecture

Visualizes the architecture of the `SentimentClassifier` model in a tree-like format, showing its components and parameter details.

* **Model type**: `SentimentClassifier`, built on top of a pretrained `BertModel` with an added linear classification head.
* **Total parameters**: \~108 million (108,311,810 parameters).
* **BERT components**:

  * **BertEmbeddings**: Includes word embeddings, position embeddings, token type embeddings, and a LayerNorm.
  * **BertEncoder**: A stack of multiple `BertLayer` blocks, each containing:

    * `BertAttention`: Multi-head self-attention with separate `query`, `key`, and `value` linear layers.
    * `BertIntermediate`: First feed-forward layer (typically expanding the hidden dimension).
    * `BertOutput`: Second feed-forward layer projecting back to the hidden size, followed by LayerNorm.
  * **BertPooler**: Pools the `[CLS]` token representation.
* **Classifier**: A `Linear` layer mapping the 768-dimensional pooled output to 2 logits for binary classification.


In [13]:
model = SentimentClassifier()
print_model(model)

🌳 SentimentClassifier<all params:108311810>
├── BertModel(bert)
│   ├── BertEmbeddings(embeddings)
│   │   ├── Embedding(word_embeddings)|weight[28996,768]
│   │   ├── Embedding(position_embeddings)|weight[512,768]
│   │   ├── Embedding(token_type_embeddings)|weight[2,768]
│   │   └── LayerNorm(LayerNorm)|weight[768]|bias[768]
│   ├── BertEncoder(encoder)
│   │   └── ModuleList(layer)
│   │       └── 💠 ]8;id=672290;/home/wukong/miniconda3/envs/genaibook/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py\BertLayer]8;;\]8;id=871065;/home/wukong/miniconda3/envs/genaibook/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py\(0-11)<🦜:7087872x12>]8;;\
│   │           ┣━━ BertAttention(attention)
│   │           ┃   ┣━━ BertSdpaSelfAttention(self)
│   │           ┃   ┃   ┗━━ 💠 ]8;id=141247;/home/wukong/miniconda3/envs/genaibook/lib/python3.10/site-packages/torch/nn/modules/linear.py\Linear]8;;\]8;id=540278;/home/wukong/miniconda3/envs/genaibook/lib/python3.10/site-packages/torch/nn/modules/linear.py\(query,key,value)<🦜:590592x3>|]8;;\]8;id=275041;/home/wukong/miniconda3/envs/genaibook/lib/python3.10/site-packages/torch/nn/modules/linear.py\weight[768,768]|]8;;\]8;id=541193;/home/wukong/miniconda3/envs/genaibook/lib/python3.10/site-packages/torch/nn/modules/linear.py\bias[768]]8;;\
│   │           ┃   ┗━━ BertSelfOutput(output)
│   │           ┃       ┣━━ Linear(dense)|weight[768,768]|bias[768]
│   │           ┃       ┗━━ LayerNorm(LayerNorm)|weight[768]|bias[768]
│   │           ┣━━ BertIntermediate(intermediate)
│   │           ┃   ┗━━ Linear(dense)|weight[3072,768]|bias[3072]
│   │           ┗━━ BertOutput(output)
│   │               ┣━━ Linear(dense)|weight[768,3072]|bias[768]
│   │               ┗━━ LayerNorm(LayerNorm)|weight[768]|bias[768]
│   └── BertPooler(pooler)
│       └── Linear(dense)|weight[768,768]|bias[768]
└── Linear(classifier)|weight[2,768]|bias[2]

# IMDBDataset — Custom PyTorch dataset class for IMDB sentiment data
- Takes a list of samples and a tokenizer.
- Converts each text into token IDs, attention masks, and token type IDs.
- Converts sentiment labels into integers (pos → 1, neg → 0).

> Run prepare_imdb_json.py to get the train and test json files.

In [ ]:
class IMDBDataset(Dataset):
    """
    Dataset class for IMDB sentiment analysis
    """
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        text = " ".join(item['text'])  # Join tokens back to text
        
        # Encode text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Convert label
        label = 1 if item['label'] == 'pos' else 0
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


def load_imdb_data():
    """
    Reads training and test data from imdb_train.json and imdb_test.json files, each line containing one JSON object.
    """
    # Load training data
    train_data = []
    with open('imdb_train.json', 'r') as f:
        for line in f:
            train_data.append(json.loads(line.strip()))
    # Load test data
    test_data = []
    with open('imdb_test.json', 'r') as f:
        for line in f:
            test_data.append(json.loads(line.strip()))
    return train_data, test_data

# Model Training Loop - Fine-Tuning

Sets up optimizer, scheduler, metrics tracking, and trains/evaluates the model for each epoch, saving the best model based on validation F1 score.

In [4]:
def train_model(model, train_loader, val_loader, device, tokenizer, num_epochs=3, learning_rate=2e-5):
    """
    Train the sentiment classifier
    """
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    
    # Learning rate scheduler
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )
    
    # Training history
    history = {
        'train_losses': [],
        'val_losses': [],
        'train_acc': [],
        'val_acc': [],
        'train_f1': [],
        'val_f1': []
    }
    
    best_val_f1 = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []
        
        for batch in tqdm(train_loader, desc='Training'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['label'].to(device)
            
            optimizer.zero_grad()
            
            logits = model(input_ids, attention_mask, token_type_ids)
            loss = F.cross_entropy(logits, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
            train_loss += loss.item()
            
            # Get predictions
            probs = F.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                labels = batch['label'].to(device)
                
                logits = model(input_ids, attention_mask, token_type_ids)
                loss = F.cross_entropy(logits, labels)
                
                val_loss += loss.item()
                
                # Get predictions
                probs = F.softmax(logits, dim=1)
                preds = torch.argmax(probs, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        # Calculate metrics
        train_acc = accuracy_score(train_labels, train_preds)
        val_acc = accuracy_score(val_labels, val_preds)
        train_f1 = f1_score(train_labels, train_preds)
        val_f1 = f1_score(val_labels, val_preds)
        
        # Store history
        history['train_losses'].append(train_loss / len(train_loader))
        history['val_losses'].append(val_loss / len(val_loader))
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)
        
        print(f'Train Loss: {history["train_losses"][-1]:.4f} | Val Loss: {history["val_losses"][-1]:.4f}')
        print(f'Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}')
        print(f'Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f}')
        print()
        
        # Save best model
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            save_model(model, tokenizer, history, '.sft_hf_bert')
    
    return history

# Save artifacts after training so they can be reused

- Saves model weights to model.pth (state_dict).
- Saves tokenizer using Hugging Face's save_pretrained() format.
- Saves the training history (loss/accuracy/F1 over epochs) to training_history.json for later visualization or debugging.


In [5]:
def save_model(model, tokenizer, history, save_dir):
    """
    Save the trained model and tokenizer
    """
    os.makedirs(save_dir, exist_ok=True)
    # Save model
    model_path = os.path.join(save_dir, 'model.pth')
    torch.save(model.state_dict(), model_path)
    # Save tokenizer
    tokenizer_path = os.path.join(save_dir, 'tokenizer')
    tokenizer.save_pretrained(tokenizer_path)
    # Save training history
    history_path = os.path.join(save_dir, 'training_history.json')
    with open(history_path, 'w') as f:
        json.dump(history, f, indent=2)
    print(f'Model saved to {save_dir}')


def load_model(model, tokenizer, save_dir):
    """
    Load the trained model and tokenizer
    """
    # Load model
    model_path = os.path.join(save_dir, 'model.pth')
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    # Load tokenizer
    tokenizer_path = os.path.join(save_dir, 'tokenizer')
    tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    return model, tokenizer

# Model Inference

- Sets model to eval() and tokenizes the input text with the same max_length used in training.
- Moves tensors to the specified device (CPU/GPU).
- Runs the model to obtain logits and converts them to probabilities via softmax.
- Selects the predicted class with argmax and its confidence (max prob).
- Returns a human-readable label ('Positive'/'Negative') and a float confidence in [0,1].

In [6]:
def predict_sentiment(model, tokenizer, text, device):
    """
    Predict sentiment for a given text
    """
    model.eval()
    # Tokenize text
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    token_type_ids = encoding['token_type_ids'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attention_mask, token_type_ids)
        probs = F.softmax(logits, dim=1)
        pred = torch.argmax(probs, dim=1)
        confidence = torch.max(probs, dim=1)[0]
    sentiment = "Positive" if pred.item() == 1 else "Negative"
    return sentiment, confidence.item()

# Orchestrate the full workflow

- Picks device (CUDA if available).
- Loads IMDB data via load_imdb_data() and splits training into train/validation (e.g., 90/10 split).
- Initializes a BertTokenizer and the SentimentClassifier model; moves model to device.
- Wraps datasets with DataLoader (batching, shuffling). Typical batch sizes for BERT fine-tuning are 8–32 depending on VRAM.
- Calls train_model() to fine-tune BERT for num_epochs with AdamW + LR schedule.
- Calls save_model() to persist weights, tokenizer, and history to '.sft_hf_bert' directory.
- Optionally runs a small list of demo texts through predict_sentiment() to print qualitative predictions and confidences.


In [9]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # Load data
    print('Loading IMDB dataset...')
    train_data, test_data = load_imdb_data()
    
    # Split training data into train and validation
    train_size = int(0.9 * len(train_data))
    val_data = train_data[train_size:]
    train_data = train_data[:train_size]
    
    print(f'Training samples: {len(train_data)}')
    print(f'Validation samples: {len(val_data)}')
    print(f'Test samples: {len(test_data)}')
    
    # Initialize tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    
    # Create datasets
    train_dataset = IMDBDataset(train_data, tokenizer)
    val_dataset = IMDBDataset(val_data, tokenizer)
    test_dataset = IMDBDataset(test_data, tokenizer)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    # Initialize model
    model = SentimentClassifier().to(device)
    
    # Check if model already exists
    if os.path.exists('.sft_hf_bert/model.pth'):
        print('Loading existing model...')
        model, tokenizer = load_model(model, tokenizer, '.sft_hf_bert')
        print('Model loaded successfully!')
    else:
        print('Training new model...')
        # Train model
        history = train_model(model, train_loader, val_loader, device, tokenizer, num_epochs=3)
        print('Training completed!')
    
    # Test the model
    print('\n=== Testing Model ===')
    test_texts = [
        "This movie is absolutely fantastic! I loved every minute of it.",
        "Terrible film, waste of time and money. Don't watch it.",
        "The acting was okay but the plot was confusing.",
        "Amazing performance by all actors, highly recommended!",
        "Boring and predictable, I fell asleep halfway through."
    ]
    
    for i, text in enumerate(test_texts, 1):
        sentiment, confidence = predict_sentiment(model, tokenizer, text, device)
        print(f'{i}. Text: {text}')
        print(f'   Prediction: {sentiment} (Confidence: {confidence:.3f})\n')

In [10]:
main()

Using device: cuda
Loading IMDB dataset...
Training samples: 22500
Validation samples: 2500
Test samples: 25000


/home/wukong/miniconda3/envs/genaibook/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training new model...
Epoch 1/3


Validation: 100%|██████████| 157/157 [00:05<00:00, 30.61it/s]


Train Loss: 0.3560 | Val Loss: 0.1737
Train Acc: 0.8434 | Val Acc: 0.9384
Train F1: 0.8214 | Val F1: 0.9682

Model saved to .sft_hf_bert
Epoch 2/3


Validation: 100%|██████████| 157/157 [00:05<00:00, 30.62it/s]


Train Loss: 0.2106 | Val Loss: 0.4315
Train Acc: 0.9203 | Val Acc: 0.8544
Train F1: 0.9105 | Val F1: 0.9215

Epoch 3/3


Validation: 100%|██████████| 157/157 [00:05<00:00, 30.73it/s]


Train Loss: 0.1222 | Val Loss: 0.6527
Train Acc: 0.9637 | Val Acc: 0.8412
Train F1: 0.9592 | Val F1: 0.9138

Training completed!

=== Testing Model ===
1. Text: This movie is absolutely fantastic! I loved every minute of it.
   Prediction: Positive (Confidence: 0.998)

2. Text: Terrible film, waste of time and money. Don't watch it.
   Prediction: Negative (Confidence: 0.999)

3. Text: The acting was okay but the plot was confusing.
   Prediction: Negative (Confidence: 0.997)

4. Text: Amazing performance by all actors, highly recommended!
   Prediction: Positive (Confidence: 0.999)

5. Text: Boring and predictable, I fell asleep halfway through.
   Prediction: Negative (Confidence: 0.993)

